# Asset pricing

In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import scipy.stats

data = pd.read_csv('input/FamaFrench.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1026 entries, 0 to 1025
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    1026 non-null   int64  
 1   VWMe    1026 non-null   float64
 2   SMB     1026 non-null   float64
 3   HML     1026 non-null   float64
 4   RF      1026 non-null   float64
 5   S1V1    1026 non-null   float64
 6   S1V2    1026 non-null   float64
 7   S1V3    1026 non-null   float64
 8   S1V4    1026 non-null   float64
 9   S1V5    1026 non-null   float64
 10  S2V1    1026 non-null   float64
 11  S2V2    1026 non-null   float64
 12  S2V3    1026 non-null   float64
 13  S2V4    1026 non-null   float64
 14  S2V5    1026 non-null   float64
 15  S3V1    1026 non-null   float64
 16  S3V2    1026 non-null   float64
 17  S3V3    1026 non-null   float64
 18  S3V4    1026 non-null   float64
 19  S3V5    1026 non-null   float64
 20  S4V1    1026 non-null   float64
 21  S4V2    1026 non-null   float64
 22  

In [6]:
data.head()

,date,VWMe,SMB,HML,RF,S1V1,S1V2,S1V3,S1V4,S1V5,...,S4V1,S4V2,S4V3,S4V4,S4V5,S5V1,S5V2,S5V3,S5V4,S5V5
0,192607,2.62,-2.16,-2.92,0.22,3.61,-3.69,-0.64,-1.42,-0.64,...,3.29,1.24,1.29,0.55,2.56,3.18,6.08,2.00,2.93,0.56
1,192608,2.56,-1.49,4.88,0.25,-1.94,-6.78,3.81,1.21,4.82,...,0.76,4.11,1.93,2.13,4.47,1.17,4.10,1.82,5.64,7.76
2,192609,0.36,-1.38,-0.01,0.23,-6.41,3.45,-5.19,3.08,0.75,...,1.87,-0.08,-1.84,1.56,2.18,-1.37,3.66,-0.23,-0.30,-2.43
3,192610,-3.43,0.04,0.71,0.32,-8.66,-10.02,-3.80,0.04,-3.06,...,0.13,-1.71,-2.33,-2.93,-5.21,-3.14,-3.13,-2.21,-4.59,-5.81
4,192611,2.44,-0.24,-0.31,0.31,3.77,12.42,2.31,-3.36,1.40,...,3.54,2.28,3.85,5.00,1.78,4.31,2.61,1.47,3.55,2.56


In [7]:
dates = data['date'].values
factors = data[['VWMe', 'SMB', 'HML']].values
riskfree = data['RF'].values
portfolios = data.iloc[:, 5:].values

factors = np.mat(factors)
riskfree = np.mat(riskfree)
portfolios = np.mat(portfolios)

In [8]:
portfolios.shape

(1026, 25)

In [9]:
factors.shape

(1026, 3)